In [ ]:
import numpy as np
import pymc as pm

from pymc_marketing.clv.distributions import continuous_contractual, continuous_non_contractual

from lifetimes.datasets import load_cdnow_summary

from scipy.special import expit, hyp2f1

import xarray as xr

import matplotlib.pyplot as plt

In [ ]:
a = 0.8
b = 2.5
alpha = 3
r = 4

rng = np.random.default_rng(seed=34)

df = load_cdnow_summary(index_col=[0])
T = df["T"].values

In [ ]:
def rng_fn(rng, a, b, r, alpha, T, T0, size):
    p = rng.beta(a, b, size=size)
    lam = rng.gamma(r, 1 / alpha, size=size)

    return continuous_contractual.rng_fn(rng, lam, p, T, T0, size=size)

In [ ]:
data = rng_fn(rng, a, b, r, alpha, T, 0, size=len(T))

In [ ]:
recency = data[..., 0]
frequency = data[..., 1]
alive = 1 - data[..., 2]

In [ ]:
def conditional_probability_alive_lifetimes(
    frequency, 
    recency, 
    T
):
    log_div = (r + frequency) * np.log((alpha + T) / (alpha + recency)) + np.log(
        a / (b + np.maximum(frequency, 1) - 1)
    )

    return np.atleast_1d(np.where(frequency == 0, 1.0, expit(-log_div)))

In [ ]:
plt.hist(conditional_probability_alive_lifetimes(frequency, recency, T), bins=40);

In [ ]:
with pm.Model() as model:
    pm.Normal("a", a, sigma=0.1)
    pm.Normal("b", b, sigma=0.1)
    pm.Normal("alpha", alpha, sigma=0.1)
    pm.Normal("r", r, sigma=0.1)
    
    trace = pm.sample(chains=2)

In [ ]:
trace

In [ ]:
dims = ("customer_id",)
coords = {"customer_id": range(len(frequency))}

frequency = xr.DataArray(
    frequency, 
    dims=dims,
    coords=coords,
)

recency = xr.DataArray(
    recency, 
    dims=dims,
    coords=coords,
)

T = xr.DataArray(
    T, 
    dims=dims,
    coords=coords,
)

In [ ]:
def conditional_probability_alive(frequency, recency, T, trace):

    dims = ("customer_id",)
    coords = {"customer_id": range(len(frequency))}
    
    to_xarray = lambda array: xr.DataArray(data=array, coords=coords, dims=dims)

    frequency = to_xarray(frequency)
    recency = to_xarray(recency)
    T = to_xarray(T)
    
    a = trace.posterior["a"]
    b = trace.posterior["b"]
    alpha = trace.posterior["alpha"]
    r = trace.posterior["r"]
    
    log_div = (r + frequency) * np.log((alpha + T) / (alpha + recency)) + np.log(
        a / (b + np.maximum(frequency, 1) - 1)
    )

    return xr.where(frequency == 0, 1.0, expit(-log_div))

In [ ]:
plt.hist(conditional_probability_alive(frequency, recency, T, trace).mean(("draw", "chain")), bins=40);

In [ ]:
trace.posterior["a"].mean()

In [ ]:
def expected_number_of_purchases(t, frequency, recency, T):
    numerator = 1 - ((alpha + T) / (alpha + T + t)) ** (r + frequency) * hyp2f1(
        r + frequency,
        b + frequency,
        a + b + frequency - 1,
        t / (alpha + T + t),
    )
    numerator *= (a + b + frequency - 1) / (a - 1)
    denominator = 1 + (frequency > 0) * (a / (b + frequency - 1)) * (
        (alpha + T) / (alpha + recency)
    ) ** (r + frequency)
    
    return numerator/denominator

In [ ]:
def to_xarray(*arrays):
    num_customers = len(arrays[0])
    dims = ("customer_id",)
    coords = {"customer_id": range(num_customers)}

    if len(arrays) == 1:
        return xr.DataArray(data=arrays[0], coords=coords, dims=dims)

    if any(len(array) != num_customers for array in arrays):
        raise ValueError("The size of input arrays must be the same.")

    return (xr.DataArray(data=array, coords=coords, dims=dims) for array in arrays)

frequency, recency, T = to_xarray(frequency, recency, T)

t = xr.DataArray(
    range(20, 40, 2),
    coords={"times": range(10)},
    dims=("times",),
)
expected_number_of_purchases(t, frequency, recency, T)

In [ ]:
to_xarray = lambda array: xr.DataArray(array,
    coords={"customer_id": range(len(array))},
    dims=("customer_id",),
)
t = to_xarray(range(20, 40, 2))
frequency = to_xarray([1, 3, 5, 7, 9]*2)
recency = to_xarray([20, 30]*5)
T = to_xarray([25, 35]*5)

In [ ]:
expected_number_of_purchases(t, frequency, recency, T)